In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('gamesDB.csv')

In [3]:
df.columns

Index(['Unnamed: 0', 'data', 'time_casa', 'time_visitante', 'resultado',
       'id_partida', 'link'],
      dtype='object')

In [4]:
df.drop(columns='Unnamed: 0', inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 6 columns):
data              4560 non-null object
time_casa         4560 non-null object
time_visitante    4560 non-null object
resultado         4560 non-null object
id_partida        4560 non-null object
link              4560 non-null object
dtypes: object(6)
memory usage: 213.8+ KB


In [6]:
df.data[0]

'2007-05-12 19:10:00'

In [7]:
df.data = pd.to_datetime(df.data)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 6 columns):
data              4560 non-null datetime64[ns]
time_casa         4560 non-null object
time_visitante    4560 non-null object
resultado         4560 non-null object
id_partida        4560 non-null object
link              4560 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 213.8+ KB


In [10]:
links = df.link

In [ ]:
i = 0
games = len(links)
for url in links:
    